## OpenAI 

For this notebook we will use the OpenAI ChatGPT models. We import the OpenAI agent and set the api_key, you will have to provide your own API key. 

In [ ]:
# Setup OpenAI Agent
import os

os.environ["OPENAI_API_KEY"] = "sk-your-key"

from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.openai import OpenAI

## Database tool

This tool connects to a database (using SQLAlchemy under the hood) and allows an Agent to query the database and get information about the tables.

We import the ToolSpec and initialize it so that it can connect to our database

In [ ]:
# Import and initialize our tool spec
from llama_index.tools.database.base import DatabaseToolSpec

db_spec = DatabaseToolSpec(
    scheme="postgresql",  # Database Scheme
    host="localhost",  # Database Host
    port="5432",  # Database Port
    user="postgres",  # Database User
    password="x",  # Database Password
    dbname="your_db",  # Database Name
)

After initializing the Tool Spec we have an instance of the ToolSpec. A ToolSpec can have many tools that it implements and makes available to agents. We can see the Tools by converting to the spec to a list of FunctionTools, using `to_tool_list`

In [ ]:
tools = db_spec.to_tool_list()
for tool in tools:
    print(tool.metadata.name)
    print(tool.metadata.description)
    print(tool.metadata.fn_schema)

load_data
load_data(query: str) -> List[llama_index.schema.Document]
Query and load data from the Database, returning a list of Documents.

        Args:
            query (str): an SQL query to filter tables and rows.

        Returns:
            List[Document]: A list of Document objects.
        
<class 'pydantic.main.load_data'>
describe_tables
describe_tables(tables: Optional[List[str]] = None) -> str

            Describes the specifed tables in the database

            Args:
                tables (List[str]): A list of table names to retrieve details about
        
<class 'pydantic.main.describe_tables'>
list_tables
list_tables() -> List[str]

            Returns a list of available tables in the database.
            To retrieve details about the columns of specfic tables, use
            the describe_tables endpoint
        
<class 'pydantic.main.list_tables'>


We can see that the database tool spec provides 3 functions for the OpenAI agent. One to execute a SQL query, one to describe a list of tables in the database, and one to list all of the tables available in the database. 

We can pass the tool list to our OpenAI agent and test it out:

In [ ]:
agent = FunctionAgent(
    tools=db_tools.to_tool_list(),
    llm=OpenAI(model="gpt-4.1"),
)

# Context to store chat history
from llama_index.core.workflow import Context
ctx = Context(agent)

At this point our Agent is fully ready to start making queries to our database:

In [ ]:
print(await agent.run("What tables does this database contain", ctx=ctx))

In [ ]:
print(await agent.run("Can you describe the messages table", ctx=ctx))

In [ ]:
print(await agent.run("Fetch the most recent message and display the body", ctx=ctx))